In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re
import numpy as np

In [3]:
def getUniqueItems(iterable): #Website gives duplicate lists of players with same xpath
    result = []
    for item in iterable:
        item = item.text
        if item not in result:
            result.append(item)
    return result

In [4]:
def goToDraft(year):    
    driver.get("https://www.nba.com/stats/draft/history/?Season=" + year)
    time.sleep(10)

    try:
        accept_cookies = driver.find_element(By.ID, "onetrust-accept-btn-handler")
        accept_cookies.click( )
        time.sleep(10)
    except:
        time.sleep(1)

In [5]:
def initializeDraftDataframe():
    titles = driver.find_elements(By.TAG_NAME, 'th')
    titles = getUniqueItems(titles)
    
    players = driver.find_elements(By.XPATH, "//td[@class='player']")
    players = getUniqueItems(players)
    
    draft_df = pd.DataFrame(columns=[t for t in titles])
    draft_df = draft_df.rename(columns=lambda x: re.sub('\n',' ',x)) #solution for removing \n in column names 
    return draft_df

In [6]:
def basicDraftData(draft_df):
    players = driver.find_elements(By.XPATH, "//td[@class='player']")
    teams = driver.find_elements(By.XPATH, "//td[@class='team']")
    affiliations = driver.find_elements(By.XPATH, "//td[@class='first text']")

    players = getUniqueItems(players)
    
    titles = driver.find_elements(By.TAG_NAME, 'th')
    titles = getUniqueItems(titles)
    
    draft = pd.DataFrame(columns=[t for t in titles])
    draft = draft.rename(columns=lambda x: re.sub('\n',' ',x)) #solution for removing \n in column names 
    
    draft[draft.columns[0]] = [p for p in players]
    draft[draft.columns[1]] = [t.text for t in teams]
    draft[draft.columns[2]] = [a.text for a in affiliations]

    draft[draft.columns[6]] = [i+1 for i in range(len(players))]
    
    draft_df = pd.concat((draft_df, draft))
    return draft_df

In [7]:
def initializePlayerDataframe(i=0):
    #must initialize player stats dataframe
    players = driver.find_elements(By.XPATH, "//td[@class='player']")
    players = getUniqueItems(players)
    clicked = False
    while not clicked:
        try:
            playerbutton = driver.find_element(By.LINK_TEXT, players[i])
            driver.execute_script("arguments[0].click();", playerbutton)
            try:
                header = driver.find_element(By.CLASS_NAME, 'Crom_headers__QJmqP')
                header = header.text.split()
                header = header[1:]
                header += ['HEIGHT', 'WEIGHT', 'POSITION']
                clicked = True
            except:
                driver.back()
                i+1
                
            
        except:
            i+1
            
        
    player_df = pd.DataFrame(columns=[i for i in header])
    driver.back()
    time.sleep(10)
    
    return player_df

In [8]:
def getPlayerStats(player, player_df,year , rookie=False):
    try:
        playerbutton = driver.find_element(By.LINK_TEXT, player)
        driver.execute_script("arguments[0].click();", playerbutton)
        time.sleep(10)
        try:
            stats = driver.find_element(By.CLASS_NAME, 'Crom_body__2rqCI')
            stats = stats.text.split()
            if not rookie:
                stats = stats[len(stats)-(len(player_df.columns)-3):]
            else:
                stats = stats[:len(stats)]
            info = driver.find_elements(By.CLASS_NAME, 'PlayerSummary_playerInfoValue__mSfou')
            info = [h.text for h in info]

            height = info[0].split('(')
            height = height[1].strip('m)')

            weight = info[1].split('(')
            weight = weight[1].strip('kg)')
            try:
                position = driver.find_element(By.XPATH, "//p[@class='t11 md:t2']")
                position = position.text.split('|')
                position = position[-1].strip()
            except:
                position='NaN'

            stats += [height, weight, position]
        except:
            stats = ['NaN']*(len(player_df.columns)-3)
            info = driver.find_elements(By.CLASS_NAME, 'PlayerSummary_playerInfoValue__mSfou')
            info = [h.text for h in info]

            height = info[0].split('(')
            height = height[1].strip('m)')

            weight = info[1].split('(')
            weight = weight[1].strip('kg)')
            try:
                position = driver.find_element(By.XPATH, "//p[@class='t11 md:t2']")
                position = position.text.split('|')
                position = position[-1].strip()
            except:
                postion = 'NaN'

            stats += [height, weight, position]
    except:
        stats = ['NaN']*len(player_df.columns)
    player_df.loc[len(player_df.index)] = stats

    goToDraft(str(year))
    time.sleep(10)

In [13]:
driver = webdriver.Chrome(ChromeDriverManager().install())
goToDraft('2001')
draft_df = initializeDraftDataframe()
player_df = initializePlayerDataframe()
print('initialized')

for year in range(2001, 2022):
    goToDraft(str(year))
    draft_df = basicDraftData(draft_df)
    players = driver.find_elements(By.XPATH, "//td[@class='player']")
    players = getUniqueItems(players)
    for player in players:
        getPlayerStats(player, player_df, year)
    print('done')


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\asimp\.wdm\drivers\chromedriver\win32\103.0.5060.134\chromedriver.exe] found in cache
C:\Users\asimp\AppData\Local\Temp/ipykernel_28180/363670212.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


initialized
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


In [14]:
draft_df

,PLAYER,TEAM,AFFILIATION,YEAR,ROUND NUMBER,ROUND PICK,OVERALL PICK
0,Kwame Brown,Washington Wizards,Glynn Academy,NaN,NaN,NaN,1
1,Tyson Chandler,Los Angeles Clippers,Dominguez,NaN,NaN,NaN,2
2,Pau Gasol,Atlanta Hawks,FC Barcelona Basquet (Spain),NaN,NaN,NaN,3
3,Eddy Curry,Chicago Bulls,Thornwood,NaN,NaN,NaN,4
4,Jason Richardson,Golden State Warriors,Michigan State,NaN,NaN,NaN,5
...,...,...,...,...,...,...,...
55,Scottie Lewis,Charlotte Hornets,Florida,NaN,NaN,NaN,56
56,Balsa Koprivica,Charlotte Hornets,Florida State,NaN,NaN,NaN,57
57,Jericho Sims,New York Knicks,Texas,NaN,NaN,NaN,58
58,RaiQuan Gray,Brooklyn Nets,Florida State,NaN,NaN,NaN,59


In [15]:
player_df

,YEAR,TEAM,GP,MIN,PTS,FGM,FGA,FG%,3PM,3PA,...,STL,BLK,PF,FP,DD2,TD3,+/-,HEIGHT,WEIGHT,POSITION
0,2001-02,WAS,57,14.3,4.5,1.6,4.3,38.7,0.0,0.0,...,0.3,0.5,1.8,11.3,2,0,-1.5,2.11,132,Center
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.13,107,Center
2,2001-02,MEM,82,36.6,17.6,6.7,13.0,51.8,0.0,0.1,...,0.5,2.1,2.4,37.3,35,0,-3.8,2.13,113,Center-Forward
3,2001-02,CHI,72,16.0,6.7,2.6,5.2,50.1,0.0,0.0,...,0.2,0.7,2.4,13.7,4,0,-3.6,2.13,134,NaN
4,2001-02,GSW,80,32.8,14.4,5.8,13.6,42.6,1.0,3.1,...,1.3,0.4,2.4,27.1,1,0,-3.4,1.98,102,Guard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,2021-22,CHA,2,3.3,0.5,0.0,0.0,0.0,0.0,0.0,...,0.5,0.0,0.0,2.8,0,0,-2.0,1.96,84,Guard
1247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1248,2021-22,NYK,41,13.5,2.2,1.0,1.3,72.2,0.0,0.0,...,0.3,0.5,1.6,9.8,1,0,1.9,2.06,113,Center
1249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
del draft_df['TEAM'] #drops duplicate and useless columns
del draft_df['YEAR']
del draft_df['ROUND NUMBER']
del draft_df['ROUND PICK']

In [37]:
draft_df = draft_df.reset_index(drop=True)

In [38]:
draft = draft_df.join(player_df, how='outer')

In [39]:
rookie = pd.read_csv("roty.csv") #csv table of all NBA rookie of the year award winners taken from basketball-reference.com
                                 #https://www.basketball-reference.com/awards/roy.html
rookielist = [p for p in rookie['Player']]
rookielist[3] = 'Luka Doncic' #In my pd dataframe Doncic has no special characters

In [40]:
draft['ROTY'] = np.where(draft1['PLAYER'].isin(rookielist), 1, 0)

In [41]:
draft

,PLAYER,AFFILIATION,OVERALL PICK,YEAR,TEAM,GP,MIN,PTS,FGM,FGA,...,BLK,PF,FP,DD2,TD3,+/-,HEIGHT,WEIGHT,POSITION,ROTY
0,Kwame Brown,Glynn Academy,1,2001-02,WAS,57,14.3,4.5,1.6,4.3,...,0.5,1.8,11.3,2,0,-1.5,2.11,132,Center,0
1,Tyson Chandler,Dominguez,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.13,107,Center,0
2,Pau Gasol,FC Barcelona Basquet (Spain),3,2001-02,MEM,82,36.6,17.6,6.7,13.0,...,2.1,2.4,37.3,35,0,-3.8,2.13,113,Center-Forward,1
3,Eddy Curry,Thornwood,4,2001-02,CHI,72,16.0,6.7,2.6,5.2,...,0.7,2.4,13.7,4,0,-3.6,2.13,134,NaN,0
4,Jason Richardson,Michigan State,5,2001-02,GSW,80,32.8,14.4,5.8,13.6,...,0.4,2.4,27.1,1,0,-3.4,1.98,102,Guard,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,Scottie Lewis,Florida,56,2021-22,CHA,2,3.3,0.5,0.0,0.0,...,0.0,0.0,2.8,0,0,-2.0,1.96,84,Guard,0
1247,Balsa Koprivica,Florida State,57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1248,Jericho Sims,Texas,58,2021-22,NYK,41,13.5,2.2,1.0,1.3,...,0.5,1.6,9.8,1,0,1.9,2.06,113,Center,0
1249,RaiQuan Gray,Florida State,59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [42]:
draft.to_csv('draft_data.csv')

In [9]:
driver = webdriver.Chrome(ChromeDriverManager().install())
goToDraft('2022')
draft_df1 = initializeDraftDataframe()
player_df1 = initializePlayerDataframe()
print('initialized')

draft_df1 = basicDraftData(draft_df1)
players = driver.find_elements(By.XPATH, "//td[@class='player']")
players = getUniqueItems(players)
for player in players:
    getPlayerStats(player, player_df1, 2022)
print('done')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.134/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\asimp\.wdm\drivers\chromedriver\win32\103.0.5060.134]
C:\Users\asimp\AppData\Local\Temp/ipykernel_7492/3615846235.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


initialized
done


In [10]:
draft_df1

,PLAYER,TEAM,AFFILIATION,YEAR,ROUND NUMBER,ROUND PICK,OVERALL PICK
0,Paolo Banchero,Orlando Magic,Duke,NaN,NaN,NaN,1
1,Chet Holmgren,Oklahoma City Thunder,Gonzaga,NaN,NaN,NaN,2
2,Jabari Smith Jr.,Houston Rockets,Auburn,NaN,NaN,NaN,3
3,Keegan Murray,Sacramento Kings,Iowa,NaN,NaN,NaN,4
4,Jaden Ivey,Detroit Pistons,Purdue,NaN,NaN,NaN,5
5,Bennedict Mathurin,Indiana Pacers,Arizona,NaN,NaN,NaN,6
6,Shaedon Sharpe,Portland Trail Blazers,Kentucky,NaN,NaN,NaN,7
7,Dyson Daniels,New Orleans Pelicans,Ignite (G League),NaN,NaN,NaN,8
8,Jeremy Sochan,San Antonio Spurs,Baylor,NaN,NaN,NaN,9
9,Johnny Davis,Washington Wizards,Wisconsin,NaN,NaN,NaN,10


In [16]:
player_df1

,AGE,GP,GS,MIN,PTS,FGM,FGA,FG%,3PM,3PA,...,DREB,REB,AST,STL,BLK,TOV,PF,HEIGHT,WEIGHT,POSITION
0,19,39,39,33.0,17.2,6.3,13.2,47.8,1.1,3.3,...,6.1,7.8,3.2,1.1,0.9,2.4,1.9,2.08,113,Forward
1,20,32,31,26.9,14.1,5.3,8.8,60.7,1.3,3.3,...,8.1,9.9,1.9,0.8,3.7,1.9,2.7,2.13,88,Center-Forward
2,19,34,34,28.8,16.9,5.4,12.6,42.9,2.3,5.5,...,6.5,7.4,2.0,1.1,1.0,1.9,2.1,2.08,100,Forward
3,20,31,4,17.9,7.2,2.7,5.4,50.6,0.5,1.7,...,3.5,5.1,0.5,0.8,1.3,0.6,1.9,2.03,102,Forward
4,19,24,12,23.2,10.7,3.7,9.3,39.9,1.0,4.0,...,2.4,3.2,1.8,0.7,0.7,1.3,1.6,1.93,88,Guard
5,19,26,12,24.9,10.8,3.4,7.2,47.1,1.5,3.5,...,3.5,4.8,1.2,0.7,0.1,1.2,1.7,2.01,95,Guard-Forward
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.98,91,Guard
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.98,90,Guard
8,19,30,1,25.1,9.2,3.3,7.0,47.4,0.8,2.7,...,4.4,6.4,1.8,1.3,0.7,1.6,2.3,2.06,104,Forward
9,19,31,0,24.3,7.0,2.8,6.3,44.1,0.5,1.2,...,3.0,4.1,1.1,1.1,0.6,1.1,2.1,1.96,88,Guard


In [17]:
del player_df1['SCHOOL'] #drops duplicate and useless columns
del draft_df1['ROUND NUMBER']
del draft_df1['ROUND PICK']

KeyError: 'SCHOOL'

In [18]:
roty_pred = draft_df1.join(player_df1, how='outer')

In [21]:
roty_pred['YEAR'] = 2022

In [22]:
roty_pred

,PLAYER,TEAM,AFFILIATION,YEAR,OVERALL PICK,AGE,GP,GS,MIN,PTS,...,DREB,REB,AST,STL,BLK,TOV,PF,HEIGHT,WEIGHT,POSITION
0,Paolo Banchero,Orlando Magic,Duke,2022,1,19,39,39,33.0,17.2,...,6.1,7.8,3.2,1.1,0.9,2.4,1.9,2.08,113,Forward
1,Chet Holmgren,Oklahoma City Thunder,Gonzaga,2022,2,20,32,31,26.9,14.1,...,8.1,9.9,1.9,0.8,3.7,1.9,2.7,2.13,88,Center-Forward
2,Jabari Smith Jr.,Houston Rockets,Auburn,2022,3,19,34,34,28.8,16.9,...,6.5,7.4,2.0,1.1,1.0,1.9,2.1,2.08,100,Forward
3,Keegan Murray,Sacramento Kings,Iowa,2022,4,20,31,4,17.9,7.2,...,3.5,5.1,0.5,0.8,1.3,0.6,1.9,2.03,102,Forward
4,Jaden Ivey,Detroit Pistons,Purdue,2022,5,19,24,12,23.2,10.7,...,2.4,3.2,1.8,0.7,0.7,1.3,1.6,1.93,88,Guard
5,Bennedict Mathurin,Indiana Pacers,Arizona,2022,6,19,26,12,24.9,10.8,...,3.5,4.8,1.2,0.7,0.1,1.2,1.7,2.01,95,Guard-Forward
6,Shaedon Sharpe,Portland Trail Blazers,Kentucky,2022,7,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.98,91,Guard
7,Dyson Daniels,New Orleans Pelicans,Ignite (G League),2022,8,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.98,90,Guard
8,Jeremy Sochan,San Antonio Spurs,Baylor,2022,9,19,30,1,25.1,9.2,...,4.4,6.4,1.8,1.3,0.7,1.6,2.3,2.06,104,Forward
9,Johnny Davis,Washington Wizards,Wisconsin,2022,10,19,31,0,24.3,7.0,...,3.0,4.1,1.1,1.1,0.6,1.1,2.1,1.96,88,Guard


In [24]:
roty_pred.to_csv('roty_pred.csv')